# Create an HTTPS ingress controller and use your own TLS certificates
Deploy the NGINX ingress controller in an Azure Kubernetes Service (AKS) cluster. Then generate your own certificates, and create a Kubernetes secret for use with the ingress route. Finally, two applications are run in the AKS cluster, each of which is accessible over a single IP address.

## Prerequisites
* latest release of Helm and have access to the ingress-nginx Helm repository.
* Azure CLI version 2.0.64 or later.

## Create an ingress controller
To create the ingress controller, use Helm to install nginx-ingress.  The ingress controller also needs to be scheduled on a Linux node. Windows Server nodes shouldn't run the ingress controller. A node selector is specified using the --set nodeSelector parameter to tell the Kubernetes scheduler to run the NGINX ingress controller on a Linux-based node. For added redundancy, two replicas of the NGINX ingress controllers are deployed with the --set controller.replicaCount parameter. To fully benefit from running replicas of the ingress controller, make sure there's more than one node in your AKS cluster.

In [1]:
# Add the ingress-nginx repository
helm repo add ingress-nginx https://kubernetes.github.io/ingress-nginx

# Use Helm to deploy an NGINX ingress controller
helm install nginx-ingress ingress-nginx/ingress-nginx \
    --set controller.replicaCount=2 \
    --set controller.nodeSelector."beta\.kubernetes\.io/os"=linux \
    --set defaultBackend.nodeSelector."beta\.kubernetes\.io/os"=linux \
    --set controller.admissionWebhooks.patch.nodeSelector."beta\.kubernetes\.io/os"=linux

"ingress-nginx" already exists with the same configuration, skipping
NAME: nginx-ingress
LAST DEPLOYED: Tue Jul  6 22:52:08 2021
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
The ingress-nginx controller has been installed.
It may take a few minutes for the LoadBalancer IP to be available.
You can watch the status by running 'kubectl --namespace default get services -o wide -w nginx-ingress-ingress-nginx-controller'

An example Ingress that makes use of the controller:

  apiVersion: networking.k8s.io/v1beta1
  kind: Ingress
  metadata:
    annotations:
      kubernetes.io/ingress.class: nginx
    name: example
    namespace: foo
  spec:
    rules:
      - host: www.example.com
        http:
          paths:
            - backend:
                serviceName: exampleService
                servicePort: 80
              path: /
    # This section is only required if TLS is to be enabled for the Ingress
    tls:
        - hosts:
            - www.example.com
   

To get the public IP address, use the following command:

In [ ]:
 kubectl get services -o wide -w nginx-ingress-ingress-nginx-controller

NAME                                     TYPE           CLUSTER-IP    EXTERNAL-IP     PORT(S)                      AGE   SELECTOR
nginx-ingress-ingress-nginx-controller   LoadBalancer   10.0.130.74   20.85.136.225   80:31389/TCP,443:32683/TCP   30s   app.kubernetes.io/component=controller,app.kubernetes.io/instance=nginx-ingress,app.kubernetes.io/name=ingress-nginx


## Generate TLS certificate
Generate a self-signed certificate with openssl.  The following example generates a 2048-bit RSA X509 certificate valid for 365 days named aks-ingress-tls.crt. The private key file is named aks-ingress-tls.key. A Kubernetes TLS secret requires both of these files.
Run the following command to create a private key file:

In [1]:
openssl req -x509 -nodes -days 365 -newkey rsa:2048 \
    -out ecl-tls.crt \
    -keyout ecl-tls.key \
    -subj "/CN=eclwatch.com/O=ecl-tls"

Generating a RSA private key
............+++++
.................+++++
writing new private key to 'ecl-tls.key'
-----


## Create Kubernetes secret for the TLS certificate
To allow Kubernetes to use the TLS certificate and private key for the ingress controller, you create and use a Secret. The secret is defined once, and uses the certificate and key file created in the previous step. You then reference this secret when you define ingress routes.
***
Use the following example to create a Secret name aks-ingress-tls:

In [2]:
kubectl create secret tls ecl-tls \
    --key ecl-tls.key \
    --cert ecl-tls.crt

secret/ecl-tls created


## Run application
An ingress controller and a Secret with your certificate have been configured.  Use helm to deploy an instance of the application eclwatch.
Create a file, and name it eclwatch-ingress.yaml

In [ ]:
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: eclwatch-ingress
  annotations:
    kubernetes.io/ingress.class: nginx
spec:
  rules:
  -  http:
      paths:
      - path: /
        pathType: Prefix
        backend:
         service:
           name: eclwatch
           port: 
             number: 8010

Create the service using the following command:

In [ ]:
$ kubectl apply -f eclwatch-ingress.yaml
ingress.networking.k8s.io/eclwatch-ingress created

## Create an ingress route
Create a file, and name it ingress-resource.yaml

In [ ]:
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: eclwatch-ingress
  annotations:
    kubernetes.io/ingress.class: nginx
    nginx.ingress.kubernetes.io/use-regex: "true"
    nginx.ingress.kubernetes.io/rewrite-target: /$2
spec:
  tls:
  - hosts: 
    - eclwatch.com
    secretName: ecl-tls
  rules:
  - host: eclwatch.com
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: eclwatch
            port:
              number: 8010

Create the ingress resource using the following command:

In [ ]:
$ kubectl apply -f ingress-resource.yaml
ingress.networking.k8s.io/eclwatch-ingress created

## Test the ingress configuration

In [4]:
curl -v -k --resolve eclwatch.com:443:20.85.136.225 https://eclwatch.com

* Added eclwatch.com:443:20.85.136.225 to DNS cache
* Hostname eclwatch.com was found in DNS cache
*   Trying 20.85.136.225:443...
* Connected to eclwatch.com (20.85.136.225) port 443 (#0)
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*   CAfile: /Users/amyma/opt/anaconda3/ssl/cacert.pem
  CApath: none
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
* TLSv1.3 (IN), TLS handshake, Server hello (2):
* TLSv1.3 (IN), TLS handshake, Encrypted Extensions (8):
* TLSv1.3 (IN), TLS handshake, Certificate (11):
* TLSv1.3 (IN), TLS handshake, CERT verify (15):
* TLSv1.3 (IN), TLS handshake, Finished (20):
* TLSv1.3 (OUT), TLS change cipher, Change cipher spec (1):
* TLSv1.3 (OUT), TLS handshake, Finished (20):
* SSL connection using TLSv1.3 / TLS_AES_256_GCM_SHA384
* ALPN, server accepted to use http/1.1
* Server certificate:
*  subject: CN=eclwatch.com; O=ecl-tls
*  start date: Jul  7 03:16:38 2021 GMT
*  expire date: Jul  7 03:16:38 2022 GMT
*  issuer: CN=eclwatch